In [2]:
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

import numpy as np
import pandas as pd
from gekko import GEKKO
from Merge import merge
from copy import deepcopy
from DPModel import action_rollout

In [3]:
def to_list(arr):
    return [elem[0] for elem in arr]

In [4]:
min_bat_state = 0.0
max_bat_state = 13.0
min_charge = -7.0
max_charge = 7.0
deg_rate = 0.0
sbr_val = 0.10
fee = 1
verbose = True

start_time='2022-06-19 00:00:00'
end_time = '2022-06-19 23:00:00'
n = len(pd.date_range(start_time, end_time,freq='H'))

In [5]:
dfA = merge('h16')
dfB = merge('h28')
dfC = merge('k28')

yieldd_A = dfA['yield'].loc[start_time:end_time].to_numpy()
yieldd_B = dfB['yield'].loc[start_time:end_time].to_numpy()
yieldd_C = dfC['yield'].loc[start_time:end_time].to_numpy()

ini_bat_state_A = 0.0
ini_bat_state_B = 10.0
ini_bat_state_C = 0.0

price = dfA['SpotPriceDKK'].loc[start_time:end_time].to_numpy()/1000

optimals = [78.8,31.9,95.8] # SHO MPC wrt initial battery value

m = GEKKO()

In [6]:
# battery states
bat_state_A = m.Array(m.Var, n+1, lb=min_bat_state, ub=max_bat_state)
bat_state_B = m.Array(m.Var, n+1, lb=min_bat_state, ub=max_bat_state)
bat_state_C = m.Array(m.Var, n+1, lb=min_bat_state, ub=max_bat_state)

# cost states
cost_state_A = m.Array(m.Var, n)
cost_state_B = m.Array(m.Var, n)
cost_state_C = m.Array(m.Var, n)

# charge action array
charge_A = m.Array(m.Var, n, lb=min_charge, ub=max_charge)
charge_B = m.Array(m.Var, n, lb=min_charge, ub=max_charge)
charge_C = m.Array(m.Var, n, lb=min_charge, ub=max_charge)

# yield array
y_A = m.Array(m.Var, n)
y_B = m.Array(m.Var, n)
y_C = m.Array(m.Var, n)

# surplus array
s_A = m.Array(m.Var, n)
s_B = m.Array(m.Var, n)
s_C = m.Array(m.Var, n)
s_t = m.Array(m.Var, n)

# market price array
p = m.Array(m.Var, n)
# grid cost
grc = m.Array(m.Var, n)

In [7]:
# define initial battery state
m.Equation(bat_state_A[0] == ini_bat_state_A)
m.Equation(bat_state_B[0] == ini_bat_state_B)
m.Equation(bat_state_C[0] == ini_bat_state_C)

# define battery dynamics
m.Equation([bat_state_A[i+1] == bat_state_A[i]*(1-deg_rate)+charge_A[i] for i in range(n)])
m.Equation([bat_state_B[i+1] == bat_state_B[i]*(1-deg_rate)+charge_B[i] for i in range(n)])
m.Equation([bat_state_C[i+1] == bat_state_C[i]*(1-deg_rate)+charge_C[i] for i in range(n)])

# define yield
m.Equations([y_A[i] == yieldd_A[i] for i in range(n)])
m.Equations([y_B[i] == yieldd_B[i] for i in range(n)])
m.Equations([y_C[i] == yieldd_C[i] for i in range(n)])

# define surplus
m.Equations([s_A[i] == y_A[i] - charge_A[i] for i in range(n)])
m.Equations([s_B[i] == y_B[i] - charge_B[i] for i in range(n)])
m.Equations([s_C[i] == y_C[i] - charge_C[i] for i in range(n)])

# define market price
m.Equations([p[i] == price[i] for i in range(n)])

# define grid interaction
m.Equations([s_t[i] == s_A[i]+s_B[i]+s_C[i] for i in range(n)])
m.Equations([grc[i] == m.if3(s_t[i], -s_t[i]*(p[i]+fee), -s_t[i]*sbr_val*p[i]) for i in range(n)])

grid_cost = sum([grc[i] for i in range(n)])

# define cost states
#m.Equation([cost_state_A[i] == m.if3(s_t[i],-1*s_A[i]*p[i],0) for i in range(n)])
#m.Equation([cost_state_B[i] == m.if3(s_t[i],-1*s_B[i]*p[i],0) for i in range(n)])
#m.Equation([cost_state_C[i] == m.if3(s_t[i],-1*s_C[i]*p[i],0) for i in range(n)])

#m.Equation([cost_state_A[i] == m.if3(s_t[i],-1*s_A[i]*p[i],-s_A[i]*sbr_val*p[i]) for i in range(n)])
#m.Equation([cost_state_B[i] == m.if3(s_t[i],-1*s_B[i]*p[i],-s_B[i]*sbr_val*p[i]) for i in range(n)])
#m.Equation([cost_state_C[i] == m.if3(s_t[i],-1*s_C[i]*p[i],-s_C[i]*sbr_val*p[i]) for i in range(n)])

m.Equation([cost_state_A[i] == m.if3(-s_t[i],-s_A[i]*sbr_val*p[i],-1*s_A[i]*p[i]) for i in range(n)])
m.Equation([cost_state_B[i] == m.if3(-s_t[i],-s_B[i]*sbr_val*p[i],-1*s_B[i]*p[i]) for i in range(n)])
m.Equation([cost_state_C[i] == m.if3(-s_t[i],-s_C[i]*sbr_val*p[i],-1*s_C[i]*p[i]) for i in range(n)])

# define pareto optimality
m.Equation(sum(cost_state_A)<=optimals[0]-20.0)
m.Equation(sum(cost_state_B)<=optimals[1]-20.0)
m.Equation(sum(cost_state_C)<=optimals[2]-20.0)

# define cummulative costs
cumm_cost_house = sum([cost_state_A[i]+cost_state_B[i]+cost_state_C[i] for i in range(n)])

# define objective
m.Obj(grid_cost)

# Solver details
m.options.MAX_ITER = 10000
m.options.IMODE = 3
solvers = [1,3,2] 
for solver in solvers:
    try:
        if verbose:
            print(f'Trying solver {solver}')
        m.options.SOLVER = solver
        m.solve(disp=verbose)
        break 
    except Exception as e:
        if verbose:
            print(f"Solver {solver} failed with error: {e}")
else:
    print("All solvers failed")
    #m.open_folder()

Trying solver 1
apm 82.211.198.107_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          822
   Intermediates:            0
   Connections  :            0
   Equations    :          655
   Residuals    :          655
 
 Number of state variables:            822
 Number of total equations: -          654
 Number of slack variables: -          195
 ---------------------------------------
 Degrees of freedom       :            -27
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      2.66 NLPi:   63 Dpth:    0 Lvs:    3 Obj:  3.94E+02 Gap:       NaN


In [8]:
of = pd.DataFrame()
#of['charge_A'] = to_list(charge_A)
#of['bat_state_A'] = to_list(bat_state_A[:len(bat_state_A)-1])
#of['charge_B'] = to_list(charge_B)
#of['bat_state_B'] = to_list(bat_state_B[:len(bat_state_B)-1])
#of['charge_C'] = to_list(charge_C)
#of['bat_state_C'] = to_list(bat_state_C[:len(bat_state_C)-1])

of['s_A'] = to_list(s_A)
of['s_B'] = to_list(s_B)
of['s_C'] = to_list(s_C)
of['s_t'] = to_list(s_t)

of['price'] = to_list(p)

of['cost_state_A'] = to_list(cost_state_A)
of['cost_state_B'] = to_list(cost_state_B)
of['cost_state_C'] = to_list(cost_state_C)


of['cumm_cost_A'] = of['cost_state_A'].cumsum()
of['cumm_cost_B'] = of['cost_state_B'].cumsum()
of['cumm_cost_C'] = of['cost_state_C'].cumsum()

of['cost_houses'] = of['cost_state_A']+of['cost_state_B']+of['cost_state_C']
of['cumm_house_cost'] = of['cost_houses'].cumsum()

of['total_cost'] = to_list(grc)
of['total_cumm_cost'] = of['total_cost'].cumsum()

of.round(4)


,s_A,s_B,s_C,s_t,price,cost_state_A,cost_state_B,cost_state_C,cumm_cost_A,cumm_cost_B,cumm_cost_C,cost_houses,cumm_house_cost,total_cost,total_cumm_cost
0,-6.2000,2.2000,-7.3000,-11.3000,1.5404,9.5504,-3.3889,11.2448,9.5504,-3.3889,11.2448,17.4064,17.4064,28.7064,28.7064
1,-12.3000,-10.3000,-12.4000,-35.0000,1.4134,17.3849,14.5581,17.5263,26.9354,11.1693,28.7711,49.4694,66.8758,84.4694,113.1758
2,2.8000,3.7000,-6.5000,-0.0000,1.2313,-3.4476,-4.5558,0.8003,23.4877,6.6135,29.5715,-7.2031,59.6727,0.0000,113.1758
3,-3.1000,0.2000,2.9000,-0.0000,1.0400,0.3224,-0.2080,-3.0159,23.8101,6.4055,26.5556,-2.9015,56.7711,0.0000,113.1758
4,-10.8000,-3.0000,-10.2134,-24.0134,0.8971,9.6891,2.6914,9.1628,33.4992,9.0969,35.7184,21.5434,78.3145,45.5568,158.7325
5,3.6000,-6.2134,2.6134,-0.0000,0.8254,-2.9715,0.5129,-2.1572,30.5277,9.6098,33.5612,-4.6159,73.6987,0.0000,158.7325
6,-6.1781,0.9134,-3.2000,-8.4647,0.8373,5.1727,-0.7648,2.6792,35.7003,8.8450,36.2405,7.0871,80.7858,15.5518,174.2843
7,-10.1000,-9.4000,-11.3000,-30.8000,0.7333,7.4059,6.8926,8.2858,43.1063,15.7376,44.5263,22.5844,103.3702,53.3844,227.6688
8,3.7000,-7.9000,4.2000,-0.0000,0.6735,-2.4918,0.5320,-2.8285,40.6145,16.2697,41.6978,-4.7882,98.5820,0.0000,227.6688
9,-4.8477,6.3000,-1.4523,-0.0000,0.7013,0.3400,-4.4185,0.1019,40.9545,11.8512,41.7997,-3.9767,94.6053,0.0000,227.6688


# Defining costs using OOP for all buyers, all sellers

In [9]:
class Buyer:
    def __init__(self, name, price, grid_price, demand):
        self.name = name
        self.price = price
        self.grid_price = grid_price
        self.demand = demand
        
        self.total_purchase = 0
        self.peer_purchase = 0
        self.grid_purchase = 0
    
    def purchase(self, quantity):
        self.peer_purchase += quantity
        self.total_purchase += quantity
    
    def unfulfilled_demand(self):
        return self.demand - self.total_purchase
    
    def purchase_from_grid(self):
        demand_gap = self.unfulfilled_demand()
        cost = demand_gap * self.grid_price
        self.grid_purchase += demand_gap
        self.total_purchase += demand_gap
        return cost
    
    def purchase_from_peers(self):
        return self.peer_purchase*self.price
    
    def total_cost(self):
        return self.purchase_from_grid()+self.purchase_from_peers()

In [10]:
class Seller:
    def __init__(self, name, price, sbr, supply):
        self.name = name
        self.price = price
        self.sbr = sbr
        self.supply = supply
        
        self.total_sale = 0
        self.peer_sale = 0
        self.grid_sale = 0
    
    def sell(self, quantity):
        self.total_sale += quantity
        self.peer_sale += quantity
    
    def unsold_supply(self):
        return self.supply - self.total_sale
    
    def sell_to_grid(self):
        unsold = self.unsold_supply()
        revenue = unsold * self.price * self.sbr
        self.total_sale += unsold
        return revenue
    
    def sell_to_peers(self):
        return self.peer_sale*self.price
    
    def total_cost(self):
        return self.sell_to_peers() + self.sell_to_grid()


In [57]:
def energy_exchange(buyers, sellers):
    if (not buyers) or (not sellers):
        return buyers

    buyer = max(buyers, key=lambda b: b.demand) # Find the buyer with the highest demand
        
    seller = max(sellers, key=lambda s: s.supply) # Find the seller with the highest supply
    
    if buyer.demand > seller.supply:
        buyer.purchase(seller.supply)
        seller.sell(seller.supply)
        sellers.remove(seller)
        return [buyer] + energy_exchange(buyers, sellers)
    
    elif seller.supply > buyer.demand:
        seller.sell(buyer.demand)
        buyer.purchase(buyer.demand)
        buyers.remove(buyer)
        return [seller] + energy_exchange(buyers, sellers)
    
    else:
        buyer.purchase(buyer.demand)
        seller.sell(seller.supply)
        buyers.remove(buyer)
        sellers.remove(seller)
        return [buyer, seller] + energy_exchange(buyers, sellers)


In [58]:
class EnergyMarket:
    def __init__(self, participants, price, grid_price):
        self.buyers = {}
        self.sellers = {}
        self.participants = participants
        self.price = price
        self.grid_price = grid_price
        
        for name, value in self.participants.items():
            if value < 0:
                buyer = Buyer(name, self.price, self.grid_price, -value)
                self.buyers[name] = buyer
            else:
                seller = Seller(name, self.price, 0.10, value)
                self.sellers[name] = seller
        
        self.buyers_list = list(self.buyers.values())
        self.sellers_list = list(self.sellers.values())
        
    def get_buyers(self):
        return self.buyers
    
    def get_sellers(self):
        return self.sellers

    def cal_costs(self):
        results = energy_exchange(self.buyers_list,self.sellers_list)
        return self.buyers, self.sellers
    
    def get_total_costs(self):
        buy_dict, sell_dict = self.cal_costs()
        costs = {}
        for name, buyer in buy_dict.items():
            costs[name] = buyer.total_cost()
        for name, seller in sell_dict.items():
            costs[name] = -seller.total_cost()
        return costs

In [59]:
nf = pd.DataFrame()
nf['A'] = to_list(s_A)
nf['B'] = to_list(s_B)
nf['C'] = to_list(s_C)

In [60]:
new = pd.DataFrame()
for i in range(len(nf)):
    thing = pd.DataFrame(EnergyMarket(nf.iloc[i].to_dict(),price[i],price[i]).get_total_costs(), index=[0])
    new = pd.concat([new,thing],ignore_index=True)

In [61]:
nf['OA'] = new['A'].to_list()
nf['OB'] = new['B'].to_list()
nf['OC'] = new['C'].to_list()
nf['cumm_OA'] = nf['OA'].cumsum()
nf['cumm_OB'] = nf['OB'].cumsum()
nf['cumm_OC'] = nf['OC'].cumsum()

In [62]:
nf

,A,B,C,OA,OB,OC,cumm_OA,cumm_OB,cumm_OC
0,-6.200000,2.200000,-7.300000,9.550418,-3.388858,11.244847,9.550418,-3.388858,11.244847
1,-12.300000,-10.300000,-12.400000,17.384943,14.558123,17.526284,26.935362,11.169265,28.771132
2,2.800000,3.700000,-6.500000,-3.447640,-4.555810,8.003450,23.487721,6.613455,36.774582
3,-3.100000,0.200000,2.900000,3.223907,-0.207994,-3.015913,26.711628,6.405461,33.758669
4,-10.800000,-3.000000,-10.213397,9.689112,2.691420,9.162847,36.400740,9.096881,42.921516
5,3.600000,-6.213397,2.613397,-2.971548,5.128724,-2.157176,33.429192,14.225605,40.764340
6,-6.178072,0.913397,-3.200000,5.172653,-0.764751,2.679232,38.601845,13.460855,43.443572
7,-10.100000,-9.400000,-11.300000,7.405926,6.892644,8.285838,46.007771,20.353499,51.729410
8,3.700000,-7.900000,4.200000,-2.491765,5.320255,-2.828490,43.516006,25.673754,48.900920
9,-4.847732,6.300000,-1.452268,3.399957,-4.418505,1.018548,46.915963,21.255249,49.919468


### Theory: Defining costs using logic/expert system

In [17]:
def round_one_decimal(number):
    """
    Returns the float that ignores everything beyond the first decimal

    Return type: float
    """
    return int(number*10)/10

In [18]:
def doub_seller(s1,s2,b,pr,grpr,sbr): # s1 and s2 want to sell, b is the solo buyer
    
    if abs(s1) >= abs(s2):            # s1 selling more than s2
        left_over = abs(s1)-abs(b)         # s1 tries to fit to the demand of b
        
        if round_one_decimal(left_over) >=0: # s1 fits the demand of b
            
            c_b = -b*pr
            c_s1 = b*pr + left_over*-pr*sbr
            c_s2 = -s2*pr*sbr
        else:                                # s2 joins s1 to fit demand of b
            
            c_s1 = -s1*pr
            
            left_over_2 = s2 - abs(left_over)
            
            if round_one_decimal(left_over_2) >= 0: # s1 and s2 matched demand of b
                c_b = -b*pr
                c_s2 = left_over_2*-pr*sbr + abs(left_over)*-pr
            
            else:                       # s2 wasn't enough
                c_b = (s1+s2)*pr + -left_over_2*grpr
                c_s2 = s2*-pr
        
        return c_s1,c_s2,c_b 
    
    else:
        c_s2,c_s1,c_b = doub_seller(s2,s1,b,pr,grpr,sbr)
        return c_s1,c_s2,c_b 

In [19]:
def solo_seller(s,b1,b2,pr,grpr,sbr): # a is the solo seller, # b is greater than or equal to c

    if abs(b1)>=abs(b2):         # B's demand is greater than C's demand
        left_over = s-abs(b1)        # A sells all to B first
        
        if round_one_decimal(left_over) >= 0:           # B buys from A & A still has something left
            c_b1 = -b1*pr
            
            left_over_2 = left_over-abs(b2) 
            
            if round_one_decimal(left_over_2) >= 0.0:    # C also buys from A & A still has something left
                c_b2 = -b2*pr
                c_s = -left_over_2*pr*sbr+(abs(b1)+abs(b2))*-pr

            else:                   # C also buys from A & has to buy more from grid 
                c_b2 = left_over*pr+abs(left_over_2)*grpr
                c_s = s*pr

        else:                       # A was not able to meet B's demand
            c_s = -s*pr
            c_b2 = -b2*grpr
            c_b1 = abs(left_over)*grpr+s*pr
        
        return c_s,c_b1,c_b2
    
    else:
        c_s,c_b2,c_b1 = solo_seller(s,b2,b1,pr,grpr,sbr)
        return c_s,c_b1,c_b2

In [20]:
A = of['s_A'].to_list()
B = of['s_B'].to_list()
C = of['s_C'].to_list()
T = of['s_t'].to_list()
price = of['price'].to_list()
grid_price = [i+1 for i in price]
sbr = 0.1

In [21]:
cA = []
cB = []
cC = []
for i in range(len(A)):
    
    if A[i]<=0 and B[i]<=0 and C[i]<=0: # all of them need to buy
        cA.append(-A[i]*grid_price[i])
        cB.append(-B[i]*grid_price[i])
        cC.append(-C[i]*grid_price[i])
    
    elif A[i]>0 and B[i]>0 and C[i]>0: # all of them need to sell
        cA.append(-A[i]*price[i]*sbr)
        cB.append(-B[i]*price[i]*sbr)
        cC.append(-C[i]*price[i]*sbr)
    
    
    elif A[i]>0 and B[i]<=0 and C[i]<=0: # A is seller and B and C are buying
        c_s,c_b1,c_b2 = solo_seller(A[i],B[i],C[i],price[i],grid_price[i],sbr)
        cA.append(c_s)
        cB.append(c_b1)
        cC.append(c_b2)
    
    elif B[i]>0 and A[i]<=0 and C[i]<=0: # B is seller and A and C are buying
        c_s,c_b1,c_b2 = solo_seller(B[i],A[i],C[i],price[i],grid_price[i],sbr)
        cA.append(c_b1)
        cB.append(c_s)
        cC.append(c_b2)
    
    elif C[i]>0 and A[i]<=0 and B[i]<=0: # C is seller and A and B are buying
        c_s,c_b1,c_b2 = solo_seller(C[i],A[i],B[i],price[i],grid_price[i],sbr)
        cA.append(c_b1)
        cB.append(c_b2)
        cC.append(c_s)
    
    elif A[i]>0 and B[i]>0 and C[i]<=0:
        c_s1,c_s2,c_b = doub_seller(A[i],B[i],C[i],price[i],grid_price[i],sbr)
        cA.append(c_s1)
        cB.append(c_s2)
        cC.append(c_b)
    
    elif A[i]>0 and C[i]>0 and B[i]<=0:
        c_s1,c_s2,c_b = doub_seller(A[i],C[i],B[i],price[i],grid_price[i],sbr)
        cA.append(c_s1)
        cB.append(c_b)
        cC.append(c_s2)
    
    else: # B[i]>0 and C[i]>0 and A[i]<=0:
        c_s1,c_s2,c_b = doub_seller(B[i],C[i],A[i],price[i],grid_price[i],sbr)
        cA.append(c_b)
        cB.append(c_s1)
        cC.append(c_s2)

In [22]:
tf = pd.DataFrame()
tf['A'] = A
tf['B'] = B
tf['C'] = C
tf['S'] = to_list(s_t)
tf['pr'] = price
tf['gpr'] = grid_price
tf['cA'] = cA
tf['cB'] = cB
tf['cC'] = cC
tf['cumm_A'] = tf['cA'].cumsum()
tf['cumm_B'] = tf['cB'].cumsum()
tf['cumm_C'] = tf['cC'].cumsum()
tf.round(3)

,A,B,C,S,pr,gpr,cA,cB,cC,cumm_A,cumm_B,cumm_C
0,-6.200,2.200,-7.300,-11.300,1.540,2.540,15.750,-3.389,16.345,15.750,-3.389,16.345
1,-12.300,-10.300,-12.400,-35.000,1.413,2.413,29.685,24.858,29.926,45.435,21.469,46.271
2,2.800,3.700,-6.500,-0.000,1.231,2.231,-3.448,-4.556,8.003,41.988,16.913,54.275
3,-3.100,0.200,2.900,-0.000,1.040,2.040,3.224,-0.208,-3.016,45.212,16.705,51.259
4,-10.800,-3.000,-10.213,-24.013,0.897,1.897,20.489,5.691,19.376,65.701,22.397,70.635
5,3.600,-6.213,2.613,-0.000,0.825,1.825,-2.972,5.129,-2.157,62.729,27.526,68.478
6,-6.178,0.913,-3.200,-8.465,0.837,1.837,10.437,-0.765,5.879,73.167,26.761,74.357
7,-10.100,-9.400,-11.300,-30.800,0.733,1.733,17.506,16.293,19.586,90.672,43.053,93.943
8,3.700,-7.900,4.200,-0.000,0.673,1.673,-2.492,5.320,-2.828,88.181,48.374,91.114
9,-4.848,6.300,-1.452,-0.000,0.701,1.701,3.400,-4.419,1.019,91.581,43.955,92.133
